# Classificador géneres de música

Com a exemple d'utilització de models que treballen en audio anem a fer un classificador de gèneres de música. Per a això farem servir el dataset GTZAN, un dataset de 1000 mostres d'àudio etiquetades amb el gènere de la música.

## Instal·lació de llibreries
Per a poder executar aquest notebook necessitarem instal·lar les següents llibreries:

In [1]:
%pip install transformers datasets librosa soundfile torch accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


## Carreguem el dataset

In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all", trust_remote_code=True)
gtzan

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

Com podem veure, el dataset consta de 99 mostres d'àudio etiquetades amb el gènere de la música.

Els audios estan en format de 22050 Hz, per a poder-los processar amb el model necessitarem convertir-los a un format que pugui ser processat pel model (normalment 16kHz). Això ho farem amb la classe `Audio` de la llibreria datasets.

In [3]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=16000))

## Creació del dataset de `test`

Per a poder avaluar el model necessitarem un dataset de test. Per a això dividirem el dataset en dos parts, una per a entrenar el model i una altra per a avaluar-lo.

In [4]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

Una vegada separat el dataset en dos parts, el dataset de test contindrà 100 mostres d'àudio.

A continuació mostrarem una mostra del dataset de test.

In [5]:
gtzan['train'][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

De cada mostra del dataset de test podem veure que tenim les següents dades:
- `audio`: El path a l'arxiu d'àudio.
- `array`: L'àudio en format d'array. El valor de cada element de l'array representa l'amplitud de l'ona en un instant de temps. Com el valor de samplig és de 16000 Hz, aquest array tindrà 16000 elements per segon.
- `genre`: El gènere de la música com a enter. Podem utilitzar el métode `int2str()` del `feature` _genre()_ per a obtenir el gènere en format llegible.

In [6]:
int2str = gtzan["train"].features["genre"].int2str
int2str(gtzan['train'][0]['genre'])

'pop'

## Testeig del model sense entrenar

Abans de començar a entrenar el model, testejarem el model sense entrenar per a veure com es comporta. Utilitzarem el model `distilhubert`, un model pre-entrenat per a classificar audio i lleuger de refinar.

Per a utilitzar el model farem servir la classe `pipeline` de la llibreria transformers.

In [7]:
from transformers import pipeline
import torch

classifier = pipeline(
    "audio-classification", model="ntu-spml/distilhubert",
    batch_size=16,
    device=torch.device('cuda')
)

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight', 'projector.weight', 'classifier.bias', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [8]:
classifier(gtzan['train'][0]['audio'])

[{'score': 0.5049870014190674, 'label': 'LABEL_0'},
 {'score': 0.4950129985809326, 'label': 'LABEL_1'}]

Anem a calcular la precisió del model sense entrenar. Per a això farem servir el dataset de test.

El primer que farem serà calcular les prediccions del model per a cada mostra del dataset de test.

A continuació mostrarem les prediccions del model per a la primera mostra del dataset de test.

In [9]:
predictions = [classifier(sample['audio']) for sample in gtzan['test']]
predictions[0]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'score': 0.5024532079696655, 'label': 'LABEL_0'},
 {'score': 0.4975467324256897, 'label': 'LABEL_1'}]

Un cop tenim les prediccions del model, les compararem amb les etiquetes reals per a calcular la precisió del model.

A continuació mostrarem la precisió del model.

In [10]:
from sklearn.metrics import accuracy_score

y_true = [f"LABEL_{sample['genre']}" for sample in gtzan['test']]
y_pred = [prediction[0]['label'] for prediction in predictions]

accuracy_score(y_true, y_pred)

0.1


## Entrenament del model

Com podem veure, el model sense entrenar té una precisió del 10%, molt poc. Això és degut a que el model no ha estat entrenat amb el dataset GTZAN.

Per a entrenar el model farem servir la classe `Trainer` de la llibreria transformers. Aquesta classe ens permet entrenar models de manera senzilla i eficient.

Mentre que amb altres models necessitem un `Tokenizer` en aquest cas farem servir un `feature_extractor`. Aquesta classe ens permetrà processar les mostres d'àudio per a convertir-les en un format que pugui ser processat pel model.

A continuació crearem el `feature_extractor` que farem servir per a entrenar el model.

In [11]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

A continuació processarrem les mostres d'àudio, convertint-les en un format que pugui ser processat pel model. En el nostre cas, retallarem les mostres d'àudio a 30 segons utilitzant les opcions `max_length` i `padding` del `feature_extractor` i llevarem les dades que no ens interessen del dataset amb el mètode `remove_columns`.

In [12]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [13]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=16,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

Renomenarem la columna `genre` a `label` per a que el `Trainer` pugui identificar-la com a columna de labels.

In [14]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

Per últim abans de començar a entrenar el model, crearem un diccionari en les correspondències entre els noms dels gèneres i els seus valors enters, per a que el `Trainer` pugui identificar-los i permetre un canvi ràpid entre els dos formats.

In [15]:
id2label = {
    str(i): int2str(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

## Entrenament del model

A continuació crearem el model que anem a entrenar.

In [16]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained(
    model_id, num_labels=len(id2label)
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight', 'projector.weight', 'classifier.bias', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


A continuació crearem els `TrainerArguments`, que ens permetrà configurar el `Trainer` per a entrenar el model.

In [17]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 3

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

A continuació crearem el `Trainer`, classe que s'encarregarà de fer l'entrenament del model.

In [18]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [19]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.795500,1.613516,0.570000
2,1.305900,1.204727,0.730000
3,1.127900,1.085730,0.750000


TrainOutput(global_step=339, training_loss=1.5500371547575194, metrics={'train_runtime': 1905.8312, 'train_samples_per_second': 1.415, 'train_steps_per_second': 0.178, 'total_flos': 1.8401964823872e+17, 'train_loss': 1.5500371547575194, 'epoch': 3.0})

In [20]:
trainer.evaluate()

{'eval_loss': 1.085729956626892,
 'eval_accuracy': 0.75,
 'eval_runtime': 50.5415,
 'eval_samples_per_second': 1.979,
 'eval_steps_per_second': 0.257,
 'epoch': 3.0}

## Utilització del model entrenat

El primer que farem serà crear el pipelineamb el model que hem entrenat per a classificar gèneres de música.

In [28]:
music_classifier = pipeline(
    "audio-classification",
    model=model,
    feature_extractor=feature_extractor,
    batch_size=16,
    device=torch.device('cuda')
)

Com alternativa podem utilitzar un model semblant al nostre, ja pre-entrenat

In [42]:
music_classifier = pipeline(
    "audio-classification",
    model="ihanif/distilhubert-music-gtzan-classification",
    batch_size=16,
    device=torch.device('cuda')
)

config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/94.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at ihanif/distilhubert-music-gtzan-classification were not used when initializing HubertForSequenceClassification: ['hubert.encoder.pos_conv_embed.conv.weight_v', 'hubert.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing HubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ihanif/distilhubert-music-gtzan-classification and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'hubert.enc

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

## Classificació de cançons

Un cop creat el noy pipeline podem fer-lo servir per a classificar una cançó. Aquest mètode rep com a paràmetre el path a la cançó que volem classificar i retorna el gènere de la cançó.

Previament necessitarem música per a poder-la classificar. A continuació teniu alguns enllaços de música lliure per a poder-la descarregar i fer servir en aquest notebook:

- [Música clàssica](https://freemusicarchive.org/genre/Classical)
- [Música electrònica](https://freemusicarchive.org/genre/Electronic)
- [Música pop](https://freemusicarchive.org/genre/Pop)
- [Música rock](https://freemusicarchive.org/genre/Rock)
- [Música jazz](https://freemusicarchive.org/genre/Jazz)

També proporcionem enllaços directes que a alguns `samples` que podeu descarregar en la llibreria requests:


In [37]:
import requests

musica = [
    "https://archive.org/download/cd_the-very-best-of-frank-sinatra_frank-sinatra-frank-nancy-sinatra/disc1/01.07.%20Frank%20Sinatra%20-%20Fly%20Me%20to%20the%20Moon%20%28In%20Other%20Words%29_sample.mp3",
    "https://archive.org/download/cd_random-access-memories_daft-punk/disc1/05.%20Daft%20Punk%20-%20Instant%20Crush_sample.mp3",
    "https://ia800101.us.archive.org/21/items/cd_debbie-does-dallas_andrew-sherman/disc1/02.%20Andrew%20Sherman%20-%20Overture_sample.mp3",
    "https://archive.org/download/cd_greatest-hits_journey/Journey%20%281988%29%20-%20Journey%27s%20Greatest%20Hits%20%5BFLAC%5D/02%20-%20Don%27t%20Stop%20Believin%27_sample.mp3",
    "https://archive.org/download/cd_generator_bad-religion/disc1/01.%20Bad%20Religion%20-%20Generator_sample.mp3",
    "https://archive.org/download/geniesduclassique_vol1no12/1-09%20Concerto%20Brandeburghese%20No.3%20-%20Adagio.mp3"
]

filenames = []

for url in musica:
    filename = url.split("/")[-1]
    filenames.append(filename)

    with open(filename, "wb") as f:
        f.write(requests.get(url).content)

filenames

['01.07.%20Frank%20Sinatra%20-%20Fly%20Me%20to%20the%20Moon%20%28In%20Other%20Words%29_sample.mp3',
 '05.%20Daft%20Punk%20-%20Instant%20Crush_sample.mp3',
 '02.%20Andrew%20Sherman%20-%20Overture_sample.mp3',
 '02%20-%20Don%27t%20Stop%20Believin%27_sample.mp3',
 '01.%20Bad%20Religion%20-%20Generator_sample.mp3',
 '1-09%20Concerto%20Brandeburghese%20No.3%20-%20Adagio.mp3']

In [43]:
for filename in filenames:
    print(f"Classificació de {filename}: {music_classifier(filename)}")

Classificació de 01.07.%20Frank%20Sinatra%20-%20Fly%20Me%20to%20the%20Moon%20%28In%20Other%20Words%29_sample.mp3: [{'score': 0.9830442070960999, 'label': 'country'}, {'score': 0.005799238570034504, 'label': 'rock'}, {'score': 0.0037795649841427803, 'label': 'blues'}, {'score': 0.003486399073153734, 'label': 'pop'}, {'score': 0.001573882531374693, 'label': 'classical'}]
Classificació de 05.%20Daft%20Punk%20-%20Instant%20Crush_sample.mp3: [{'score': 0.9724063277244568, 'label': 'disco'}, {'score': 0.011590911075472832, 'label': 'pop'}, {'score': 0.009916543029248714, 'label': 'rock'}, {'score': 0.002160327974706888, 'label': 'reggae'}, {'score': 0.0016030846163630486, 'label': 'metal'}]
Classificació de 02.%20Andrew%20Sherman%20-%20Overture_sample.mp3: [{'score': 0.9287788271903992, 'label': 'rock'}, {'score': 0.020959066227078438, 'label': 'classical'}, {'score': 0.019138414412736893, 'label': 'metal'}, {'score': 0.011989741586148739, 'label': 'blues'}, {'score': 0.007792058400809765, '